In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import shutil

# json -> labeling image

In [2]:
def make_mask_training(label_building, label_road):

    # 도로
    if label_road == meta_road:
        mask_road = np.zeros((1024, 1024, 3), dtype=np.uint8)
        road_coords = []

        for r in range(len(label_road["features"])):
            road = label_road["features"][r]["properties"]
            temp = road["road_imcoords"].split(",")
            if len(temp) <= 1: continue
            coords = np.array([int(round(float(c))) for c in temp]).reshape(-1, 2)
            road_coords.append(coords)

    mask_road = cv2.fillPoly(mask_road, road_coords, color=(255,255,255))

    # 빌딩
    if label_building == meta_building:
        mask_building = np.zeros((1024, 1024, 3), dtype=np.uint8)
        build_coords = []

        for r in range(len(label_building["features"])):
            building = label_building["features"][r]["properties"]
            temp = building["building_imcoords"].split(",")
            if len(temp) <= 1: continue
            coords = np.array([int(round(float(c))) for c in temp]).reshape(-1, 2)
            build_coords.append(coords)

    mask_building = cv2.fillPoly(mask_road, build_coords, color=(200,200,200))

    plt.imshow(mask_building)
    plt.imshow(mask_road,alpha=0.7) 


    cv2.imwrite(f'./LV2_training_set/labels/{i[:-5]}.png', mask_building)  # 저장

    return mask_building, mask_road

In [3]:
def make_mask_validation(label_building, label_road):

    # 도로
    if label_road == meta_road:
        mask_road = np.zeros((1024, 1024, 3), dtype=np.uint8)
        road_coords = []

        for r in range(len(label_road["features"])):
            road = label_road["features"][r]["properties"]
            temp = road["road_imcoords"].split(",")
            if len(temp) <= 1: continue
            coords = np.array([int(round(float(c))) for c in temp]).reshape(-1, 2)
            road_coords.append(coords)

    mask_road = cv2.fillPoly(mask_road, road_coords, color=(255,255,255))
    
    # 빌딩
    if label_building == meta_building:
        mask_building = np.zeros((1024, 1024, 3), dtype=np.uint8)
        build_coords = []

        for r in range(len(label_building["features"])):
            building = label_building["features"][r]["properties"]
            temp = building["building_imcoords"].split(",")
            if len(temp) <= 1: continue
            coords = np.array([int(round(float(c))) for c in temp]).reshape(-1, 2)
            build_coords.append(coords)

    mask_building = cv2.fillPoly(mask_road, build_coords, color=(200,200,200))

    plt.imshow(mask_building)
    plt.imshow(mask_road,alpha=0.7) 


    cv2.imwrite(f'./LV2_validation_set/labels/{i[:-5]}.png', mask_building)  # 저장

    return mask_building, mask_road

## training label 생성

In [ ]:
file_list = os.listdir('./LV2_training_set/labels_json/building')
file_list_building = [file for file in file_list if file.endswith('.json')]

file_list = os.listdir('./LV2_training_set/labels_json/road')
file_list_road = [file for file in file_list if file.endswith('.json')]

print(f'빌딩 : {len(file_list_building)}')
print(f'도로 : {len(file_list_road)}')

file_building_road = file_list_building + file_list_road

os.mkdir('./LV2_training_set/labels') # labels 디렉토리 생성

for i in file_building_road:
    with open(f"./LV2_training_set/labels_json/building/{i}", "r") as jfile_building, open(f"./LV2_training_set/labels_json/road/{i}", "r") as jfile_road:
        meta_building, meta_road = json.load(jfile_building), json.load(jfile_road)
        make_mask_training(meta_building, meta_road)

## validation label 생성

In [ ]:
file_list = os.listdir('./LV2_validation_set/labels_json/building')
file_list_building = [file for file in file_list if file.endswith('.json')]

file_list = os.listdir('./LV2_validation_set/labels_json/road')
file_list_road = [file for file in file_list if file.endswith('.json')]

print(f'빌딩 : {len(file_list_building)}')
print(f'도로 : {len(file_list_road)}')

file_building_road = file_list_building + file_list_road

os.mkdir('./LV2_validation_set/labels') # labels 디렉토리 생성

for i in file_building_road:
    with open(f"./LV2_validation_set/labels_json/building/{i}", "r") as jfile_building, open(f"./LV2_validation_set/labels_json/road/{i}", "r") as jfile_road:
        meta_building, meta_road = json.load(jfile_building), json.load(jfile_road)
        make_mask_validation(meta_building, meta_road)

# training (512x512) resize

In [ ]:
# image resize
data_path = './LV2_training_set/images'
img_list_dir = os.listdir(data_path)
img_list = [file for file in img_list_dir if file.endswith('.png')]

os.mkdir('./LV2_training_set/images_4')

print(len(img_list))

for x in img_list:
    img_read = cv2.imread(f'{data_path}/{x}')

    cut1 = img_read[0:512, 0:512, :]
    cut2 = img_read[0:512, 512:1024, :]
    cut3 = img_read[512:1024, 0:512, :]
    cut4 = img_read[512:1024, 512:1024, :]

    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_1.png' , cut1)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_2.png' , cut2)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_3.png' , cut3)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_4.png' , cut4)

In [ ]:
# label resize
data_path = './LV2_training_set/labels'
img_list_dir = os.listdir(data_path)
img_list = [file for file in img_list_dir if file.endswith('.png')]

os.mkdir('./LV2_training_set/labels_4')

print(len(img_list))
for x in img_list:
    img_read = cv2.imread(f'{data_path}/{x}')

    cut1 = img_read[0:512, 0:512, :]
    cut2 = img_read[0:512, 512:1024, :]
    cut3 = img_read[512:1024, 0:512, :]
    cut4 = img_read[512:1024, 512:1024, :]

    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_1.png' , cut1)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_2.png' , cut2)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_3.png' , cut3)
    cv2.imwrite(f'{data_path}_4/' + f'{x[:-4]}_4.png' , cut4)

# 검정[0,0,0] 배경 제거

In [ ]:
data_path = './LV2_training_set/labels_4'
img_list_dir = os.listdir(data_path)
img_list = [file for file in img_list_dir if file.endswith('.png')]

color_list = []
compare_img = cv2.imread('./background_img.png')

for x in img_list:
    target_img = cv2.imread(f'{data_path}/{x}')
    diff_img = compare_img - target_img

    F_break = True

    for i in range(0,len(diff_img)):
        for j in range(0,len(diff_img)):
            if not np.array_equal(np.array([0, 0, 0]),diff_img[i][j]):
                color_list.append(x)
                F_break = False
                break    
        if(F_break == False):
            break

black_list = list(set(img_list) - set(color_list))
print(len(black_list))

In [9]:
data_path_labels_4 = './LV2_training_set/labels_4'
data_path_images_4 = './LV2_training_set/images_4'

for i in black_list:
    img_label = f'{data_path_labels_4}/{i}'
    img_image = f'{data_path_images_4}/{i}'

    os.remove(img_label)
    os.remove(img_image)

In [ ]:
shutil.rmtree('./LV2_training_set/labels') # labels 디렉토리 제거
print('전처리 완료')